In [1]:
import pandas
import separator
data =pandas.read_csv("FAOSTAT_DispoAlim.csv")
df=pandas.read_csv("FAOSTAT_DispoAlimKg.csv")
population=pandas.read_csv("population.csv")
ListeAnimaux=pandas.read_csv("ListeAnimaux.csv")
ListeVegetaux=pandas.read_csv("ListeVegtaux.csv")
def conversion(population,valeur):
    return (population*valeur)/1000

def NatureProduit(produit):
    if produit in ListeAnimaux.values:
        return "Animale"
    elif produit in ListeVegetaux.values:
        return "Vegetale"
    else:
        return "Produit ni végétal ni animal"
    

In [2]:
data=data.drop(columns=(["Code Domaine","Domaine","Code Élément","Code Année"]))

In [3]:
dispo_alim_kcal_p_j=data.groupby("Élément").get_group("Disponibilité alimentaire (Kcal/personne/jour)")

In [4]:
dispo_alim_kcal_p_j=dispo_alim_kcal_p_j.drop(columns=(["Élément","Code Pays"]))

In [5]:
dispo_prot=data.groupby("Élément").get_group("Disponibilité de protéines en quantité (g/personne/jour)")

In [6]:
dispo_prot=dispo_prot.drop(columns=(["Élément","Code Pays"]))

In [7]:
dispo_mat_gr=data.groupby("Élément").get_group("Disponibilité de matière grasse en quantité (g/personne/jour)")

In [8]:
dispo_mat_gr=dispo_mat_gr.drop(columns=(["Élément","Code Pays"]))

In [9]:
df=df.drop(columns=["Code Domaine","Domaine","Code Élément","Élément","Code Pays","Code Année"])

In [10]:

population=population.drop(columns="Unnamed: 0")

In [11]:
population=population.rename(columns={"Valeur":"Population"})

In [12]:
CalculPoidsDispo=pandas.merge(population,df,on=["Pays","Année"])

In [13]:
CalculPoidsDispo=CalculPoidsDispo.assign(dispo_alim_tonnes=conversion(CalculPoidsDispo.Population,CalculPoidsDispo.Valeur))

In [14]:
CalculPoidsDispo=CalculPoidsDispo.drop(columns=(["Population","Valeur"]))

In [15]:
LaBase=data.drop(columns=["Élément","Valeur"]).drop_duplicates()

In [16]:
Merge1=pandas.merge(LaBase,dispo_alim_kcal_p_j,on=["Code Produit","Pays","Produit","Année"])

In [17]:
Merge1=Merge1.rename(columns={"Valeur":"dispo_alim_kcal_p_j"})

In [18]:
Merge2=pandas.merge(Merge1,dispo_prot,on=["Code Produit","Pays","Produit","Année"],how="outer")

In [19]:
Merge2=Merge2.rename(columns={"Valeur":"dispo_prot"})

In [20]:
Merge3=pandas.merge(Merge2,dispo_mat_gr,on=["Code Produit","Pays","Produit","Année"], how ="outer")

In [21]:
Merge3=Merge3.rename(columns={"Valeur":"dispo_mat_gr"})

In [22]:
TableFinale=pandas.merge(Merge3,CalculPoidsDispo,on=["Code Pays","Pays","Code Produit","Produit","Année"],how="outer")

In [23]:
TableFinale

,Code Pays,Pays,Code Produit,Produit,Année,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr,dispo_alim_tonnes
0,2,Afghanistan,2511,Blé,2010,1386.0,37.32,5.01,4605019.68
1,2,Afghanistan,2511,Blé,2011,1383.0,37.29,4.73,4710644.25
2,2,Afghanistan,2511,Blé,2012,1378.0,37.16,4.64,4809877.75
3,2,Afghanistan,2511,Blé,2013,1369.0,36.91,4.69,4895346.96
4,2,Afghanistan,2805,Riz (Eq Blanchi),2010,159.0,3.04,0.30,442440.84
5,2,Afghanistan,2805,Riz (Eq Blanchi),2011,167.0,3.19,0.31,475575.70
6,2,Afghanistan,2805,Riz (Eq Blanchi),2012,146.0,2.79,0.27,425602.75
7,2,Afghanistan,2805,Riz (Eq Blanchi),2013,141.0,2.70,0.27,422228.64
8,2,Afghanistan,2513,Orge,2010,19.0,0.57,0.17,60203.76
9,2,Afghanistan,2513,Orge,2011,22.0,0.67,0.20,71598.30


In [24]:
NatureDesProduits = []
for valeur in TableFinale.Produit:
    NatureDesProduits.append(NatureProduit(valeur))
    
TableFinale["Nature"]=NatureDesProduits

In [25]:
dataframe=pandas.DataFrame(NatureDesProduits)

In [26]:
ListeVegetaux=ListeVegetaux.drop(columns="Unnamed: 0")

In [27]:
TableFinale

,Code Pays,Pays,Code Produit,Produit,Année,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr,dispo_alim_tonnes,Nature
0,2,Afghanistan,2511,Blé,2010,1386.0,37.32,5.01,4605019.68,Vegetale
1,2,Afghanistan,2511,Blé,2011,1383.0,37.29,4.73,4710644.25,Vegetale
2,2,Afghanistan,2511,Blé,2012,1378.0,37.16,4.64,4809877.75,Vegetale
3,2,Afghanistan,2511,Blé,2013,1369.0,36.91,4.69,4895346.96,Vegetale
4,2,Afghanistan,2805,Riz (Eq Blanchi),2010,159.0,3.04,0.30,442440.84,Vegetale
5,2,Afghanistan,2805,Riz (Eq Blanchi),2011,167.0,3.19,0.31,475575.70,Vegetale
6,2,Afghanistan,2805,Riz (Eq Blanchi),2012,146.0,2.79,0.27,425602.75,Vegetale
7,2,Afghanistan,2805,Riz (Eq Blanchi),2013,141.0,2.70,0.27,422228.64,Vegetale
8,2,Afghanistan,2513,Orge,2010,19.0,0.57,0.17,60203.76,Vegetale
9,2,Afghanistan,2513,Orge,2011,22.0,0.67,0.20,71598.30,Vegetale


In [37]:
Merge2

,Code Pays,Pays,Code Produit,Produit,Année,dispo_alim_kcal_p_j,dispo_prot
0,2,Afghanistan,2511,Blé,2010,1386.0,37.32
1,2,Afghanistan,2511,Blé,2011,1383.0,37.29
2,2,Afghanistan,2511,Blé,2012,1378.0,37.16
3,2,Afghanistan,2511,Blé,2013,1369.0,36.91
4,2,Afghanistan,2805,Riz (Eq Blanchi),2010,159.0,3.04
5,2,Afghanistan,2805,Riz (Eq Blanchi),2011,167.0,3.19
6,2,Afghanistan,2805,Riz (Eq Blanchi),2012,146.0,2.79
7,2,Afghanistan,2805,Riz (Eq Blanchi),2013,141.0,2.70
8,2,Afghanistan,2513,Orge,2010,19.0,0.57
9,2,Afghanistan,2513,Orge,2011,22.0,0.67


In [32]:
LaBase.reset_index()

,index,Code Pays,Pays,Code Produit,Produit,Année
0,0,2,Afghanistan,2511,Blé,2010
1,1,2,Afghanistan,2511,Blé,2011
2,2,2,Afghanistan,2511,Blé,2012
3,3,2,Afghanistan,2511,Blé,2013
4,12,2,Afghanistan,2805,Riz (Eq Blanchi),2010
5,13,2,Afghanistan,2805,Riz (Eq Blanchi),2011
6,14,2,Afghanistan,2805,Riz (Eq Blanchi),2012
7,15,2,Afghanistan,2805,Riz (Eq Blanchi),2013
8,24,2,Afghanistan,2513,Orge,2010
9,25,2,Afghanistan,2513,Orge,2011
